In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#a new type of import
import pandas_profiling as pp

sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
#bring thte data to the dataframe
data = pd.read_csv('/kaggle/input/learn-together/train.csv.zip')

#describe the data
data.describe()

In [ ]:
#as id is not a feature or something, we can drop it 
data.drop(['Id'], axis = 1 , inplace = True)

data.describe()

In [ ]:
print(data.columns)

Feature selection using chi^2 method

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

y = data['Cover_Type']
X = data.drop(['Cover_Type'] , axis = 1)
X = X.apply(lambda x : abs(x))
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=25)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Features','Score']  #naming the dataframe columns
print(featureScores.nlargest(25,'Score'))  #print 10 best features

Result though not impressive, but we can try out by selcting features from it in version 2

In [ ]:
#using Pandas_profiling for eda
report = pp.ProfileReport(data)
report.to_file("report.html")

report

In [ ]:
data = data.rename(columns={"Horizontal_Distance_To_Roadways": "HDR", "Horizontal_Distance_To_Fire_Points": "HDF"  })
data = data.rename(columns={"Horizontal_Distance_To_Hydrology": "HDH", "Vertical_Distance_To_Hydrology": "VDH" })

In [ ]:
data.columns

In [ ]:
sns.heatmap(data.corr() , cmap = 'gist_ncar')

In [ ]:
#appending all the positive correlated data
lst = []
column = []

for col in data.columns:
    if(data.corr()['Cover_Type'][col] > 0.0):
        column.append(col)
        print(col +" : " +  str(data.corr()['Cover_Type'][col]))
        lst.append(float(data.corr()['Cover_Type'][col]))
#print(lst.sort()[:5])

In [ ]:
print(type(column[1]))
print(type(lst[0]))

In [ ]:
print(column[0:16])
column = column[:-1]

In [ ]:
column

In [ ]:
selected = data[column]

test_df = pd.read_csv('/kaggle/input/learn-together/test.csv.zip')
test_df = test_df.rename(columns={"Horizontal_Distance_To_Roadways": "HDR", "Horizontal_Distance_To_Fire_Points": "HDF"  })
test_df = test_df.rename(columns={"Horizontal_Distance_To_Hydrology": "HDH", "Vertical_Distance_To_Hydrology": "VDH" })
test_sel = test_df[["Id"] + column]

In [ ]:
test_sel.columns

In [ ]:
#selected.describe()
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.metrics import accuracy_score

y = data['Cover_Type'][:]

X_train , X_test , y_train , y_test = train_test_split(selected , y , stratify = y , random_state = 7)

In [ ]:
X_test.shape
y_test = y_test.to_numpy()

In [ ]:
from sklearn.ensemble import AdaBoostClassifier , BaggingClassifier , RandomForestClassifier , ExtraTreesClassifier , GradientBoostingClassifier
from sklearn.tree import ExtraTreeClassifier

In [ ]:
#AdaBoostClassifier

l = [10 , 50 , 25 , 75 , 100 , 150 , 200]

for n in range(5 , 100 ,5):
    clf = AdaBoostClassifier(n_estimators=n , random_state = 0)
    clf.fit(X_train , y_train)
    
    y_pred = clf.predict(X_test)
      
    print("For l=" + str(n) , end = ' ')
    print(accuracy_score(y_pred , y_test))

In [ ]:
#ExtraTreesClassifier
for n in range(300 , 500 ,10):
    clf = ExtraTreesClassifier(n_estimators=n , random_state = 0)
    clf.fit(X_train , y_train)
    
    y_pred = clf.predict(X_test)
      
    print("For Max Depth " + str(n) , end = ' ')
    print(accuracy_score(y_pred , y_test))

In [ ]:
%%time
#GradientBoostingClassifier
for n in range(300 , 500 ,50):
    clf = GradientBoostingClassifier(learning_rate=0.2 , n_estimators=n , random_state = 0)
    clf.fit(X_train , y_train)
    
    y_pred = clf.predict(X_test)
      
    print("For Max Depth " + str(n) , end = ' ')
    print(accuracy_score(y_pred , y_test))

In [ ]:
%%time
#RandomForestClassifier
for n in range(100 , 1500 ,100):
    clf = RandomForestClassifier(n_estimators=n , random_state = 0)
    clf.fit(X_train , y_train)
    
    y_pred = clf.predict(X_test)
      
    print("For Max Depth " + str(n) , end = ' ')
    print(accuracy_score(y_pred , y_test))

In [ ]:
#BagggingClasssifier with ExtraTreeClassifier as base estimator

base = ExtraTreeClassifier(max_depth = 500)

for n in range(100 , 1500 ,100):
    clf = BaggingClassifier(base_estimator = base , n_estimators=n , random_state = 0)
    clf.fit(X_train , y_train)
    
    y_pred = clf.predict(X_test)
      
    print("For Max Depth " + str(n) , end = ' ')
    print(accuracy_score(y_pred , y_test))

as the initial model Max Depth 200 with this expt gives the best possible accuracy
without an oversampling, undersampling,or any other optimization done for the differential distribution of train and test

In [ ]:
train_X = selected.to_numpy()
train_y = y.to_numpy()

In [ ]:
model = BaggingClassifier(base_estimator = base , n_estimators=200 , random_state = 0)

id = test_sel['Id']

test_sel = test_sel.drop(['Id'] , axis = 1)
print("Training")
model.fit(train_X , train_y)
print("Finished!!!")
print("Predicting")
pred = model.predict(test_sel)


In [ ]:
submission = pd.DataFrame({ 'Id': id,
                            'Cover_Type': pred })
submission.to_csv("submission_example.csv", index=False)